In [1]:
# !wget https://huggingface.co/datasets/mesolitica/embedding-pair-mining/resolve/main/b.cari.com.my-dataset-bge-test.jsonl

In [2]:
# !pip3 install openai

In [3]:
# we use OpenAI Azure, replace it with standard OpenAI

In [4]:
import os

os.environ['OPENAI_API_KEY'] = ''

In [5]:
import json
import openai
from tqdm import tqdm

In [6]:
with open('c.cari.com.my-dataset-bge-test.sample.json') as fopen:
    rev_data = json.load(fopen)

In [7]:
with open('openai-c.cari.com.my.vectors.json') as fopen:
    vectors = json.load(fopen)

In [9]:
vectors = {}
for k, v in tqdm(rev_data.items()):
    try:
        o = openai.Embedding.create(input=k, engine=engine)
        vectors[k] = o['data'][0]['embedding']
    except:
        pass

100%|██████████| 15545/15545 [1:07:06<00:00,  3.86it/s]


In [8]:
no_string = {no: k for no, (k, v) in enumerate(vectors.items())}
string_no = {v: k for k, v in no_string.items()}
no_string[0]

'回复 5# 当猫爱上狗  我是27号要用的， 我想应该是来不及了。。真的是很谢谢你的回复。。'

In [11]:
with open('openai-c.cari.com.my.vectors.json', 'w') as fopen:
    json.dump(vectors, fopen)

In [9]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [13]:
line = 0
positive, negative = [], []
with open('c.cari.com.my-dataset-bge-test.jsonl') as fopen:
    for l in tqdm(fopen):
        l = json.loads(l)
        query = l['query'].strip()
        v_query = vectors.get(query)
        if v_query is None:
            continue
        v_query = np.array(v_query).reshape((1, -1))
        for s in l['pos']:
            v_s = vectors.get(s.strip())
            if v_s is None:
                continue
            v_s = np.array(v_s).reshape((1, -1))
            positive.append(cosine_similarity(v_query, v_s)[0, 0])
        
        for s in l['neg']:
            v_s = vectors.get(s.strip())
            if v_s is None:
                continue
            v_s = np.array(v_s).reshape((1, -1))
            negative.append(1 - cosine_similarity(v_query, v_s)[0, 0])
            
        line += 1
        if line >= 1000:
            break

999it [00:09, 103.71it/s]


In [14]:
np.mean(positive), np.mean(negative)

(0.8173745331635356, 0.3100609159718768)

In [10]:
np_vectors = np.array(list(vectors.values()))

In [14]:
tops = {
    1: 0,
    3: 0,
    5: 0,
    10: 0,
}
total = 0
with open('c.cari.com.my-dataset-bge-test.jsonl') as fopen:
    for l in tqdm(fopen):
        l = json.loads(l)
        query = l['query'].strip()
        query_no = string_no.get(query)
        if query_no is None:
            continue
        for s in l['pos']:
            s = s.strip()
            v_s = vectors.get(s)
            s_no = string_no.get(s)
            if v_s is None:
                continue
            v_s = np.array(v_s).reshape((1, -1))
            argsort = np.argsort(cosine_similarity(v_s, np_vectors)[0])[::-1]
            for k in tops.keys():
                if s_no in argsort[:k]:
                    k_ = k + 1
                else:
                    k_ = k
                if query_no in argsort[:k_]:
                    tops[k] += 1
            total += 1

7087it [20:34,  5.74it/s] 


In [15]:
tops, total

({1: 949, 3: 2422, 5: 3155, 10: 4065}, 11324)

In [16]:
for k, v in tops.items():
    print(k, v / total)

1 0.08380430943129637
3 0.21388202048746027
5 0.27861179795125396
10 0.3589720946661957
